**ANN Project**

In [1]:
#import tensorflow as tf
#from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Prepping the dataset for training the model

#### Cleaning milage, price, and dropping some columns

In [2]:
# Load the dataset
used_cars = pd.read_csv('used_cars.csv')

used_cars = used_cars.drop(columns=['int_col', 'clean_title'])

print("Before dropping rows with NaN values:")
print(used_cars.shape[0])

used_cars = used_cars.dropna()

print("After dropping rows with NaN values:")
print(used_cars.shape[0])

used_cars['price'] = used_cars['price'].str.replace('$', '').str.replace(',', '').astype(int)
used_cars['milage'] = used_cars['milage'].str.replace('mi.', '').str.replace(',', '').astype(int)

used_cars.head(10)

Before dropping rows with NaN values:
4009
After dropping rows with NaN values:
3730


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,accident,price
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,At least 1 accident or damage reported,10300
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,At least 1 accident or damage reported,38005
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,None reported,54598
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,None reported,15500
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,None reported,34999
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,F,Silver,None reported,14798
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,None reported,31000
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,None reported,7300
8,Lexus,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,None reported,41927
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,Automatic,Fuji White,None reported,73897


# Cleaning the Accident Column

In [3]:
print(used_cars['accident'].unique())

def clean_accident(text):
    if pd.isna(text):
        return None
    elif "None reported" in text:
        return 0
    else:
        return 1
    
used_cars["accident_reported"] = used_cars["accident"].apply(clean_accident)
used_cars = used_cars.dropna(subset=["accident_reported"])
used_cars = used_cars.drop(columns=["accident"])

print(used_cars['accident_reported'].value_counts(dropna=False))

used_cars.head(10)

['At least 1 accident or damage reported' 'None reported']
accident_reported
0    2752
1     978
Name: count, dtype: int64


,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,price,accident_reported
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,10300,1
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,38005,1
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,54598,0
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,15500,0
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,34999,0
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,F,Silver,14798,0
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,31000,0
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,7300,0
8,Lexus,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,41927,0
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,Automatic,Fuji White,73897,0


In [ ]:
def clean_transmission(text):
    if pd.isna(text):
        return None
    elif 'automatic' in text.lower() or 'a/t' in text.lower():
        return 0
    elif 'manual' in text.lower() or 'm/t' in text.lower():
        return 1
    else:
        return 2

# Apply the function to clean the transmission column
used_cars["transmission"] = used_cars["transmission"].apply(clean_transmission)

# Drop rows with NaN values in the 'transmission' column
used_cars = used_cars.dropna(subset=["transmission"])


used_cars.head(10)

['6-Speed A/T' '8-Speed Automatic' 'Automatic' '7-Speed A/T' 'F' 'A/T'
 '8-Speed A/T' 'Transmission w/Dual Shift Mode' '9-Speed Automatic'
 '6-Speed M/T' '10-Speed A/T' '9-Speed A/T' '5-Speed A/T' 'Automatic CVT'
 '7-Speed Automatic with Auto-Shift' 'CVT-F'
 '6-Speed Automatic with Auto-Shift' 'M/T' '10-Speed Automatic'
 'CVT Transmission' '4-Speed A/T' '6-Speed Automatic' '4-Speed Automatic'
 '8-Speed Automatic with Auto-Shift' '5-Speed Automatic' '8-SPEED AT'
 '1-Speed Automatic' '5-Speed M/T' 'Manual' '6-Speed Manual' 'Variable'
 '2' '7-Speed Manual' 'Automatic, 9-Spd 9G-Tronic' 'Automatic, 8-Spd'
 'Automatic, 8-Spd Sport w/Sport & Manual Modes' '–'
 'Auto, 6-Spd w/CmdShft' 'Transmission Overdrive Switch'
 '7-Speed Automatic' 'Automatic, 8-Spd PDK Dual-Clutch'
 'Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Manual Modes'
 'Automatic, 8-Spd Dual-Clutch' '6-Speed' 'Automatic, 10-Spd' '7-Speed'
 '7-Speed M/T' 'Automatic, 7-Spd S tronic Dual-Clutch' 'Manual, 6-Spd'
 '6-Speed Elect

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,price,accident_reported
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,0,Black,10300,1
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,0,Moonlight Cloud,38005,1
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,0,Blue,54598,0
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,0,Black,15500,0
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,0,Glacier White Metallic,34999,0
5,Acura,ILX 2.4L,2016,136397,Gasoline,2.4 Liter,2,Silver,14798,0
6,Audi,S3 2.0T Premium Plus,2017,84000,Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,0,Blue,31000,0
7,BMW,740 iL,2001,242000,Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,0,Green,7300,0
8,Lexus,RC 350 F Sport,2021,23436,Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,0,Black,41927,0
10,Land,Rover Range Rover Sport 3.0 Supercharged HST,2021,27608,Gasoline,V6,0,Fuji White,73897,0
